In [3]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기
#Step 1. 필요한 모듈을 로딩합니다                  
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import pandas as pd 
import time

#Step 2. 사용자에게 검색 관련 정보들을 입력 받고 3. 파일 이름 정하기
query_txt = input("검색어를 입력하세요: ")
collect_cnt = int(input("검색할 건수를 입력하세요: "))
# query_txt ='빅데이터'
# collect_cnt = 20
ft_name = "c:\\py_temp\\riss.txt"
fc_name = "c:\\py_temp\\riss.csv"
fx_name = "c:\\py_temp\\riss.xls"

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
driver = webdriver.Chrome()
driver.get('https://www.riss.kr/')

time.sleep(5)
driver.maximize_window()

main = driver.window_handles
for handle in main:
    if handle != main[0]:
        driver.switch_to.window(handle)
        driver.close()

driver.switch_to.window(driver.window_handles[0])

#Step 5. 자동으로 검색어 입력 후 6 학위논문 클릭
driver.find_element(By.ID,'query').send_keys(query_txt+ '\n')
driver.find_element(By.LINK_TEXT,'학위논문').click()
time.sleep(2)

#Step 7.Beautiful Soup 로 본문 내용만 추출하기
soup_1 = BeautifulSoup(driver.page_source, 'html.parser')
content_1 = soup_1.find('div','srchResultListW').find_all('li')

#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math
total_cnt = soup_1.find('div','searchBox pd').find('span','num').get_text()
print('키워드 %s (으)로  %s 건 검색' %(query_txt,total_cnt))
collect_page_cnt = math.ceil(collect_cnt / 10)


#Step 9. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = [ ]        #번호 저장
title2 = [ ]     #논문제목 저장
writer2 = [ ]    #논문저자 저장
org2 = [ ]       #소속기관 저장
no = 1

for a in range(1, collect_page_cnt + 1) :
    time.sleep(2)
    soup_2 = BeautifulSoup(driver.page_source, 'html.parser')
    content_2 = soup_2.find('div','srchResultListW').find_all('li')
    
    for b in content_2 : 
        #1. 논문제목 있을 경우만
        try :
            title = b.find('p','title').get_text()
        except :
            continue
        else :
            f = open(ft_name, 'a' , encoding="UTF-8")
            
            print('1.번호:',no)
            no2.append(no)
            f.write('\n'+'1.번호:' + str(no))

            print('2.논문제목:',title)
            title2.append(title)
            f.write('\n' + '2.논문제목:' + title)
            
            writer = b.find('span','writer').get_text()
            print('3.저자:',writer)
            writer2.append(writer)
            f.write('\n' + '3.저자:' + writer)

            org = b.find('span','assigned').get_text()
            print('4.소속기관:' , org)
            org2.append(org)
            f.write('\n' + '4.소속기관:' + org + '\n')
            
            f.close( )
            
            no += 1
            print("\n")
            
            if no > collect_cnt :
                break

            time.sleep(1)        # 페이지 변경 전 1초 대기 

    a += 1 
    b = str(a)

    try :
        driver.find_element(By.LINK_TEXT ,'%s' %b).click() 
    except :
        driver.find_element(By.LINK_TEXT,'다음 페이지로').click()
        
print("요청하신 작업이 모두 완료되었습니다")

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['저자']=pd.Series(writer2)
df['소속(발행)기관']=pd.Series(org2)

# xls 형태로 저장하기
df.to_excel(fx_name, index=False, engine='openpyxl')

# csv 형태로 저장하기
df.to_csv(fc_name, index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')

키워드 딥러닝 (으)로  6,056 건 검색
1.번호: 1
2.논문제목: 딥러닝을 이용한 해석 가능한 함수 추정에 관한 연구
3.저자: 송낙윤
4.소속기관: 명지대학교 대학원


1.번호: 2
2.논문제목: 딥러닝과 Dilation-Erosion 기법을 접목한 영상 기반 정밀 균열 평가 기술
3.저자: 배숙경
4.소속기관: 서울시립대학교 일반대학원


1.번호: 3
2.논문제목: 딥러닝 및 관련 기법을 활용한 철근 가격 장단기예측
3.저자: 이용성
4.소속기관: 건국대학교 대학원


1.번호: 4
2.논문제목: 딥러닝 알고리즘을 이용한 건설공사 공기예측의 적정성 분석
3.저자: 안현주
4.소속기관: 경상대학교 대학원


1.번호: 5
2.논문제목: 딥러닝 기반 객체 분류에 의한 토지사용 현황 분석에 관한 연구 - 창원시 사례를 중심으로 -
3.저자: 김순한
4.소속기관: 서울시립대학교 대학원


1.번호: 6
2.논문제목: 딥러닝 모델 기반 스케일업 LULUCF 매트릭스 구축에 관한 연구
3.저자: 박정묵
4.소속기관: 강원대학교 대학원


1.번호: 7
2.논문제목: 딥러닝을 이용한 일별 천연가스 수요 예측
3.저자: 김헌주
4.소속기관: 성균관대학교 일반대학원


1.번호: 8
2.논문제목: PVD를 활용한 딥러닝 기반 도심부 링크 통행속도 추정 알고리즘 개발
3.저자: 박건규
4.소속기관: 명지대학교 대학원


1.번호: 9
2.논문제목: 딥러닝 구름 탐지 기반 NOAA/AVHRR 해수면온도 정확도 향상 연구
3.저자: 최도영
4.소속기관: 서울대학교 대학원


1.번호: 10
2.논문제목: ARM Compute Library와 Neural Networks API를 이용한 임베디드 딥러닝 시스템 개발
3.저자: 이정헌
4.소속기관: 인천대학교 일반대학원


1.번호: 11
2.논문제목: 인공지능 딥러닝 소개와 보안 동향
3.저자: 박용병
4.소속기관: 공주대학교 일반대학원


1.번호: 12
2.논문제목: 딥러닝을 이용한 시계열 데이터  군집화에 관한 연